In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tsfresh

%matplotlib inline

In [21]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, \
    load_robot_execution_failures
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()

In [22]:
timeseries

,id,time,F_x,F_y,F_z,T_x,T_y,T_z
0,1,0,-1,-1,63,-3,-1,0
1,1,1,0,0,62,-3,-1,0
2,1,2,-1,-1,61,-3,0,0
3,1,3,-1,-1,63,-2,-1,0
4,1,4,-1,-1,63,-3,-1,0
...,...,...,...,...,...,...,...,...
1315,88,10,-10,2,39,-21,-24,5
1316,88,11,-11,2,38,-24,-22,6
1317,88,12,-12,3,23,-24,-24,5
1318,88,13,-13,4,26,-29,-27,5


In [23]:
from tsfresh import extract_features
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")

Feature Extraction: 100%|██████████| 30/30 [00:06<00:00,  4.88it/s]


In [24]:
extracted_features

,F_x__variance_larger_than_standard_deviation,F_x__has_duplicate_max,F_x__has_duplicate_min,F_x__has_duplicate,F_x__sum_values,F_x__abs_energy,F_x__mean_abs_change,F_x__mean_change,F_x__mean_second_derivative_central,F_x__median,...,T_z__fourier_entropy__bins_2,T_z__fourier_entropy__bins_3,T_z__fourier_entropy__bins_5,T_z__fourier_entropy__bins_10,T_z__fourier_entropy__bins_100,T_z__permutation_entropy__dimension_3__tau_1,T_z__permutation_entropy__dimension_4__tau_1,T_z__permutation_entropy__dimension_5__tau_1,T_z__permutation_entropy__dimension_6__tau_1,T_z__permutation_entropy__dimension_7__tau_1
1,0.0,0.0,1.0,1.0,-14.0,14.0,0.142857,0.000000,-0.038462,-1.0,...,NaN,NaN,NaN,NaN,NaN,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
2,0.0,1.0,1.0,1.0,-13.0,25.0,1.000000,0.000000,-0.038462,-1.0,...,0.562335,0.562335,1.073543,1.494175,2.079442,0.937156,1.234268,1.540306,1.748067,1.831020
3,0.0,0.0,1.0,1.0,-10.0,12.0,0.714286,0.000000,-0.038462,-1.0,...,0.661563,1.039721,1.386294,1.732868,2.079442,1.265857,1.704551,2.019815,2.163956,2.197225
4,0.0,1.0,1.0,1.0,-6.0,16.0,1.214286,-0.071429,-0.038462,0.0,...,0.562335,0.735622,1.073543,1.494175,2.079442,1.156988,1.907284,2.397895,2.302585,2.197225
5,0.0,0.0,0.0,1.0,-9.0,17.0,0.928571,-0.071429,0.038462,-1.0,...,0.562335,0.562335,0.900256,1.320888,2.079442,1.156988,1.863680,2.271869,2.302585,2.197225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,1.0,1.0,0.0,1.0,-1073.0,96833.0,7.142857,-5.428571,-0.038462,-98.0,...,0.376770,0.735622,0.735622,0.735622,1.386294,1.585771,2.253858,2.397895,2.302585,2.197225
85,1.0,0.0,1.0,1.0,143.0,1683.0,1.357143,1.071429,0.076923,8.0,...,0.376770,0.376770,0.735622,0.735622,1.667462,1.332245,1.589027,1.893788,2.163956,2.197225
86,1.0,0.0,0.0,0.0,961.0,83497.0,9.071429,9.071429,0.807692,52.0,...,0.376770,0.376770,0.735622,1.073543,1.732868,0.687092,0.983088,1.159589,1.227529,1.303092
87,1.0,1.0,0.0,1.0,4509.0,1405437.0,12.928571,12.214286,-1.038462,338.0,...,0.376770,0.376770,0.735622,0.735622,1.386294,0.535961,0.836988,1.159589,1.497866,1.581094


In [2]:
# read dataframe from covid_explore.ipynb output
covid = pd.read_csv("./data/covid/covid19_20210104_mod.csv", index_col='date', parse_dates=True)
# select columns of interest and remove all dates up to March 1
# from the start of the pandemic in BC
covid_mod = covid.loc['2020-07-01':,('days_elapse', 'new_count_update')].copy()
#covid_mod.reset_index(inplace=True, drop=True)
covid_mod.columns = ['days_elapse', 'daily_case']

# add a 3, 5, 7 and 14-day rolling mean
# also add a 14-day rolling std-dev in the event a 14-day standardization is used
covid_mod['daily_case_3mean'] = covid_mod['daily_case'].rolling(window = 3).mean()
covid_mod['daily_case_5mean'] = covid_mod['daily_case'].rolling(window = 5).mean()
covid_mod['daily_case_7mean'] = covid_mod['daily_case'].rolling(window = 7).mean()
covid_mod['daily_case_14mean'] = covid_mod['daily_case'].rolling(window = 14).mean()
covid_mod['daily_case_14stdev'] = covid_mod['daily_case'].rolling(window = 14).std()

In [3]:
covid_test = covid_mod[['daily_case_7mean']].dropna().copy()

In [46]:
#covid_test['id'] = 1

In [50]:
#covid_test.reset_index(inplace=True)

In [66]:
covid_test

,daily_case_7mean
date,
2020-07-07,10.571429
2020-07-08,11.428571
2020-07-09,12.571429
2020-07-10,15.142857
2020-07-11,16.571429
...,...
2020-12-27,493.428571
2020-12-28,480.857143
2020-12-29,472.142857


In [19]:
extracted_features = extract_features(covid_test, column_id = 'id', column_sort = 'date', column_value = 'daily_case_7mean')

Feature Extraction: 100%|██████████| 1/1 [00:04<00:00,  4.50s/it]


In [20]:
extracted_features

,daily_case_7mean__variance_larger_than_standard_deviation,daily_case_7mean__has_duplicate_max,daily_case_7mean__has_duplicate_min,daily_case_7mean__has_duplicate,daily_case_7mean__sum_values,daily_case_7mean__abs_energy,daily_case_7mean__mean_abs_change,daily_case_7mean__mean_change,daily_case_7mean__mean_second_derivative_central,daily_case_7mean__median,...,daily_case_7mean__fourier_entropy__bins_2,daily_case_7mean__fourier_entropy__bins_3,daily_case_7mean__fourier_entropy__bins_5,daily_case_7mean__fourier_entropy__bins_10,daily_case_7mean__fourier_entropy__bins_100,daily_case_7mean__permutation_entropy__dimension_3__tau_1,daily_case_7mean__permutation_entropy__dimension_4__tau_1,daily_case_7mean__permutation_entropy__dimension_5__tau_1,daily_case_7mean__permutation_entropy__dimension_6__tau_1,daily_case_7mean__permutation_entropy__dimension_7__tau_1
1,1.0,0.0,0.0,1.0,47512.285714,2.445600e+07,8.661017,2.669088,0.041396,120.428571,...,0.061047,0.061047,0.121969,0.121969,0.243433,1.325435,2.05376,2.635926,3.070739,3.431099


In [4]:
test = pd.Series(covid_test['daily_case_7mean'], index = covid_test.index)

In [5]:
df_rolled1 = tsfresh.utilities.dataframe_functions.make_forecasting_frame(test, kind=1, max_timeshift=14, rolling_direction=1)
df_rolled2 = tsfresh.utilities.dataframe_functions.make_forecasting_frame(test, kind=1, max_timeshift=7, rolling_direction=1)

Rolling: 100%|██████████| 30/30 [00:02<00:00, 13.23it/s]


In [6]:
df_rolled1_mod = df_rolled1[0].groupby('id').filter(lambda x: len(x) >= 14)
df_rolled2_mod = df_rolled2[0].groupby('id').filter(lambda x: len(x) >= 7)

In [23]:
extracted_features1 = tsfresh.extract_features(df_rolled1_mod, column_id = 'id', column_sort = 'time', column_value = 'value')
extracted_features2 = tsfresh.extract_features(df_rolled2_mod, column_id = 'id', column_sort = 'time', column_value = 'value')

Feature Extraction: 100%|██████████| 29/29 [00:03<00:00,  7.95it/s]


In [24]:
extracted_features1 = extracted_features1.droplevel(0).add_suffix('_14')
extracted_features2 = extracted_features2.droplevel(0).add_suffix('_7')

In [25]:
extracted_features_combo = pd.concat([extracted_features1, extracted_features2], axis = 1, join='inner')

In [26]:
extracted_features_combo

,value__variance_larger_than_standard_deviation_14,value__has_duplicate_max_14,value__has_duplicate_min_14,value__has_duplicate_14,value__sum_values_14,value__abs_energy_14,value__mean_abs_change_14,value__mean_change_14,value__mean_second_derivative_central_14,value__median_14,...,value__fourier_entropy__bins_2_7,value__fourier_entropy__bins_3_7,value__fourier_entropy__bins_5_7,value__fourier_entropy__bins_10_7,value__fourier_entropy__bins_100_7,value__permutation_entropy__dimension_3__tau_1_7,value__permutation_entropy__dimension_4__tau_1_7,value__permutation_entropy__dimension_5__tau_1_7,value__permutation_entropy__dimension_6__tau_1_7,value__permutation_entropy__dimension_7__tau_1_7
2020-07-21,1.0,0.0,0.0,0.0,258.571429,5.071551e+03,1.219780,1.219780,0.035714,19.642857,...,0.562335,0.562335,0.562335,1.039721,1.039721,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
2020-07-22,1.0,0.0,0.0,0.0,276.571429,5.776122e+03,1.318681,1.318681,0.041667,19.928571,...,0.562335,0.562335,0.562335,0.562335,1.386294,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
2020-07-23,1.0,0.0,0.0,0.0,295.571429,6.571408e+03,1.373626,1.373626,-0.029762,20.214286,...,0.562335,0.562335,0.562335,1.039721,1.386294,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
2020-07-24,1.0,0.0,0.0,0.0,314.714286,7.419163e+03,1.274725,1.274725,-0.005952,20.500000,...,0.562335,0.562335,0.562335,1.039721,1.386294,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
2020-07-25,1.0,0.0,0.0,0.0,331.142857,8.186612e+03,1.175824,1.153846,-0.065476,21.714286,...,0.562335,0.562335,1.039721,1.039721,1.386294,0.500402,0.562335,0.636514,0.693147,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,1.0,0.0,0.0,0.0,8625.142857,5.365544e+06,15.703297,-12.780220,-1.886905,636.500000,...,0.562335,0.562335,0.562335,1.039721,1.039721,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
2020-12-28,1.0,0.0,0.0,0.0,8450.142857,5.162219e+06,15.153846,-14.692308,-0.095238,605.571429,...,0.562335,0.562335,0.562335,0.562335,1.386294,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
2020-12-29,1.0,0.0,0.0,0.0,8246.571429,4.925001e+06,15.615385,-15.153846,-0.648810,583.642857,...,0.562335,0.562335,0.562335,1.039721,1.386294,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
2020-12-30,1.0,0.0,0.0,0.0,8040.857143,4.688429e+06,16.054945,-16.054945,-0.065476,569.357143,...,0.562335,0.562335,1.039721,1.039721,1.386294,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
